# Habituation to turbulent backgrounds as a function of model rates

For IBCM, measure the robustness of alignment to one odor using, for each neuron, the difference between its maximum and second largest (or minimum?) alignments, or some similar metric. Then average across neurons to measure the full network convergence. 

We do not check new odor recognition performance for the sake of simplicity; we can infer whether the IBCM model would perform well or not based on its alignment specificity and noise. 

Things to check:
 - Convergence of IBCM as as function of $\mu$ (or equivalently the OSN input amplitude? Or some moment of the background? not sure) and $\tau_\Theta$. 
 - Convergence as a function of the number of odors
 - Convergence as a function of the strength of turbulence, whatever that means?

## Imports

In [ ]:
import numpy as np
from scipy import sparse, special
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json
from os.path import join as pj
import sys
if ".." not in sys.path:
    sys.path.insert(1, "..")


from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_hgammas_hbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor
)
from modelfcts.checktools import (
    analyze_pca_learning, 
    check_conc_samples_powerlaw_exp1
)
from modelfcts.backgrounds import (
    update_powerlaw_times_concs, 
    logof10, 
    sample_ss_conc_powerlaw, 
    generate_odorant,
    mean_turbulent_concs
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_average,
    powerlaw_cutoff_inverse_transform
)
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_hbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard, l2_norm

# Initialization

### Aesthetic parameters

In [ ]:
do_save_plots = False
do_save_outputs = False

root_dir = pj("..")
outputs_folder = pj(root_dir, "results", "for_plots", "convergence")
panels_folder = pj(root_dir, "figures", "convergence")
params_folder = pj(root_dir, "results", "common_params")

# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

# Background generation and initialization functions

In [ ]:
def linear_combi(concs, backs):
    """ concs: shaped [..., n_odors]
        backs: 2D array, shaped [n_odors, n_osn]
    """
    return concs.dot(backs)

In [ ]:
# Global choice of background and odor mixing functions
update_fct = update_powerlaw_times_concs
combine_fct = linear_combi

In [ ]:
# We will later explore the effect of varying these parameters on the convergence, 
# but put the default ones in a function
def default_background_params(n_comp):
    """ Default time and concentration parameters for the turbulent process"""
    # Turbulent background parameters: same rates and constants for all odors
    back_pms_turbulent = [
        np.asarray([1.0] * n_comp),        # whiff_tmins
        np.asarray([500.] * n_comp),       # whiff_tmaxs
        np.asarray([1.0] * n_comp),        # blank_tmins
        np.asarray([800.0] * n_comp),      # blank_tmaxs
        np.asarray([0.6] * n_comp),        # c0s
        np.asarray([0.5] * n_comp),        # alphas
    ]
    return back_pms_turbulent

In [ ]:
# Background initialization, given parameters and a seeded random generator
def initialize_given_background(back_pms, rgen, n_comp, n_dim):
    # Initial values of background process variables (t, c for each variable)
    init_concs = sample_ss_conc_powerlaw(*back_pms[:-1], size=1, rgen=rgen)
    init_times = powerlaw_cutoff_inverse_transform(
                    rgen.random(size=n_comp), *back_pms[2:4])
    tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

    # Initial background vector: combine odors with the tc_init concentrations
    back_comps = back_pms[-1]
    init_bkvec = combine_fct(tc_init[:, 1], back_comps)
    # background random variables are first in the list of initial values
    init_back = [tc_init, init_bkvec]
    
    return init_back

# IBCM simulation functions

In [ ]:
# Analyze to establish convergence to specific fixed points. 
def analyze_ibcm_simulation(sim_results, ibcm_rates_loc, back_pms, 
                            skp_loc=20, dt=1.0, duration_loc=360000.0):
    """
    Args:
        sim_results = (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
            hbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm)
        ibcm_rates_loc: learnrate_ibcm, tau_avg_ibcm, coupling_eta_ibcm, ...
            
    Returns:
        alignment_gaps, indexed [neuron]
        specif_gammas, indexed [neuron]
        gamma_vari, indexed [neuron, component]
    """
    coupling = ibcm_rates_loc[2]
    (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
        hbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm) = sim_results
    # Calculate hgammas_bar and mbars
    transient = int(5/6*duration_loc / dt) // skp_loc
    basis = back_pms[-1]
    
    # Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
    mbarser, c_gammas, hbars_gamma = compute_mbars_hgammas_hbargammas(mser_ibcm, coupling, basis)
    hbars_gamma_mean = np.mean(hbars_gamma[transient:], axis=0)
    # Sorted odor indices, from min to max, of odor alignments for each neuron
    aligns_idx_sorted = np.argsort(hbars_gamma_mean, axis=1) 
    specif_gammas = np.argmax(hbars_gamma_mean, axis=1)
    assert np.all(specif_gammas == aligns_idx_sorted[:, -1])
    
    
    # Gap between first and second largest alignments for each neuron
    n_i = hbars_gamma_mean.shape[0]
    alignment_gaps = (hbars_gamma_mean[np.arange(n_i), specif_gammas]
                     - hbars_gamma_mean[np.arange(n_i), aligns_idx_sorted[:, -2]])
    
    # Variance (fluctuations) of hbars gamma in the last 20 minutes of the simul
    # Increases when the learning rate increases
    last_steps = int(2.0*duration_loc/3.0 / dt) // skp_loc
    hbars_gamma_vari = np.var(hbars_gamma[last_steps:], axis=0)
    
    return hbars_gamma, alignment_gaps, specif_gammas, hbars_gamma_vari

In [ ]:
def run_analyze_ibcm_one_back_seed(
        ibcm_rates_loc, back_rates, inhib_rates_loc, 
        options_loc, dimensions, seedseq, minit_scale=0.2,
        duration_loc=360000.0, dt_loc=1.0, skp_loc=20, full_returns=False
    ):
    """ Given IBCM model rates and background parameters except
    background odors (but incl. number odors, c0), and a main seed sequence, 
    run and analyze convergence of IBCM on the background generated from that seed. 
    The seedseq should itself have been spawned from a root seed to have a distinct
    one per run; this still makes seeds reproducible yet distinct for different runs. 
    The seedseq here is spawned again for a background gen. seed and a simul. seed. 
    
    Args:
        dimensions: gives [n_components, n_dimensions, n_i_ibcm]
    
    Returns:
        iff full_return:
            gaps, specifs, hgamvari, hgammas_ser, sim_results
        else:
            gaps, specifs, hgamvari, None, None
        alignment_gaps: indexed [neuron]
        specif_gammas: indexed [neuron]
        gamma_vari: indexed [neuron, component]
    """
    #print("Initializing IBCM simulation...")
    # Get dimensions
    n_comp, n_dim, n_i = dimensions
    
    # Spawn back. generation seed and simul seed
    initseed, simseed = seedseq.spawn(2)
    
    # Duplicate back params before appending locally-generated odor vectors to them
    back_pms_loc = list(back_rates)
    
    # Create background
    rgen_init = np.random.default_rng(initseed)
    back_comps_loc = generate_odorant((n_comp, n_dim), rgen_init)
    back_comps_loc = back_comps_loc / l2_norm(back_comps_loc, axis=1)[:, None]

    # Add odors to the list of background parameters
    back_pms_loc.append(back_comps_loc)

    # Initialize background with the random generator with seed rgenseed
    rgen_init = np.random.default_rng(initseed)
    init_back = initialize_given_background(back_pms_loc, rgen_init, n_comp, n_dim)

    # Initial synaptic weights: small positive noise
    lambd_loc = ibcm_rates_loc[3]
    init_synapses_ibcm = minit_scale*rgen_init.standard_normal(size=[n_i, n_dim])*lambd_loc
    
    # Run the IBCM simulation
    #print("Running IBCM simulation...")
    tstart = perf_counter()
    sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back, 
                ibcm_rates_loc, inhib_rates_loc, back_pms_loc, 
                duration_loc, dt_loc, seed=simseed, 
                noisetype="uniform",  skp=skp_loc, **options_loc
    )
    tend = perf_counter()
    #print("Finished IBCM simulation in {:.2f} s".format(tend - tstart))
    
    # Now analyze IBCM simul for convergence
    #print("Starting to analyze IBCM simulation...")
    tstart = perf_counter()
    hgammas_ser, gaps, specifs, hgamvari = analyze_ibcm_simulation(sim_results, 
                        ibcm_rates_loc, back_pms_loc, skp_loc=skp_loc, duration_loc=duration_loc)
    tend = perf_counter()
    #print("Finished analyzing IBCM simulation")
    
    # Doesn't return full c gamma series, only the summary statistics of convergence
    if full_returns:
        hgammas_ser_ret = hgammas_ser
        sim_results_ret = sim_results
    else:
        hgammas_ser_ret = None
        sim_results_ret = None
    
    return gaps, specifs, hgamvari, hgammas_ser_ret, sim_results_ret

# Plotting functions

In [ ]:
# Plotting functions for IBCM
def plot_ibcm_results(res_ibcm_raw, hbars_gamma, skp=20):
    # tseries, bk_series, bkvec_series, m_series,
    # hbar_series, theta_series, w_series, y_series
    tser, bkser, bkvecser, _, _, _, _, yser = res_ibcm_raw
    tser_scaled = tser *  10.0 / 60.0  # in min
    # Plot of hbars gamma series
    fig1 , ax1, _ = plot_hbars_gamma_series(tser_scaled, hbars_gamma, 
                            skp=skp, transient=320000 // skp)
    fig1.tight_layout()
    leg = ax1.legend(loc="upper left", bbox_to_anchor=(1., 1.))
    ax1.set_xlabel("Time (min)")

    # Plot of background inhibition
    fig2, ax2, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser, bkvecser, yser, skp=skp)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax2.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax2.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig2.tight_layout()
    
    return fig1, ax1, fig2, ax2

# IBCM simulation with proper rates 
To illustrate convergence

In [ ]:
# Common parameters for all simulations
# Dimensions: 25 is enough?
n_dimensions = 25
n_components = 3  # try with 3 for simplicity by default

# Inhibition W learning and decay rates
inhib_rates_default = [0.0001, 0.00002]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration and integration time step
duration = 360000.0
deltat = 1.0

# Saving every skp simulation point, 50 is enough for plots, 
# here use 20 to get convergence time accurately
skp_default = 20 * int(1.0 / deltat)
tser_common = np.arange(0.0, duration, deltat*skp_default)

# Common model options
activ_function = "identity"  #"ReLU"

## IBCM default parameters

In [ ]:
# IBCM model parameters
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Default model rates
learnrate_ibcm = 0.00125 #5e-5
tau_avg_ibcm = 1600  # 2000
coupling_eta_ibcm = 0.6/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates_default = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

In [ ]:
# Test run for now
# Create a default background for testing purposes
meta_seedseq = np.random.SeedSequence(0x36d8487b210fe0277493149915aba4f3)

# Package dimensions and back. parameters
back_rates_default = default_background_params(n_components)
# Try changing background rates here if desired
# Concentration scale c0: multiply up-down to change convergence dynamics
# just as well as the learning rate, although with a different scaling. 
#back_rates_default[4][:] = 0.6
#back_rates_default[1][:] = 500.0  # whiff duration
#back_rates_default[3][:] = 800.0  # blank duration
dimensions_ibcm = [n_components, n_dimensions, n_i_ibcm]

# Run and analyze simulation derived from the meta seedsequence
all_res = run_analyze_ibcm_one_back_seed(ibcm_rates_default, back_rates_default, inhib_rates_default, 
                        ibcm_options, dimensions_ibcm, meta_seedseq,
                        duration_loc=duration, dt_loc=deltat, skp_loc=skp_default, full_returns=True)

gaps_def, specifs_def, hgamvari_def, hgammas_ser_def, ibcm_results_def = all_res

In [ ]:
# Visualize convergence dynamics first
plot_ibcm_results(ibcm_results_def, hgammas_ser_def, skp=skp_default)
plt.show()
plt.close()

In [ ]:
# Print some other convergence analysis results
print("{} out of {} odors covered".format(np.unique(specifs_def).size, n_components))
print("Variance of the largest hbar_gamma for each neuron:\n", hgamvari_def[np.arange(n_i_ibcm), specifs_def])

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(n_i_ibcm), gaps_def, marker="o", mfc="w", ms=8)
for i in range(n_i_ibcm):
    ax.annotate(str(specifs_def[i]), xy=(i, gaps_def[i]), ha="center", va="center")
ax.set_ylim([0.0, gaps_def.max()*1.1])
ax.set(ylabel="Alignment gap", xlabel="Neuron", xticks=np.arange(0, n_i_ibcm, 3))
plt.show()

# Simulation with learning rate too high

The neurons still select one odor, but have large oscillations around the specific fixed point. 

In [ ]:
# Simulation with excessively high learning rate
learnrate_ibcm_hi = 0.005
tau_avg_ibcm_hi = 2000
ibcm_rates_hi = [
    learnrate_ibcm_hi, 
    tau_avg_ibcm_hi, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]

# Test run for now
# Create a default background for testing purposes
meta_seedseq_hi = np.random.SeedSequence(0xa5ad7857493a6fc471b0283b8db69268)

# Package dimensions and back. parameters
back_rates_default = default_background_params(n_components)
dimensions_ibcm = [n_components, n_dimensions, n_i_ibcm]

# Run and analyze simulation derived from the meta seedsequence
all_res = run_analyze_ibcm_one_back_seed(ibcm_rates_hi, back_rates_default, inhib_rates_default, 
                        ibcm_options, dimensions_ibcm, meta_seedseq_hi,
                        duration_loc=duration, dt_loc=deltat, skp_loc=skp_default, full_returns=True)

gaps_hi, specifs_hi, hgamvari_hi, hgammas_ser_hi, ibcm_results_hi = all_res

In [ ]:
# Visualize convergence dynamics first
plot_ibcm_results(ibcm_results_hi, hgammas_ser_hi, skp=skp_default)
plt.show()
plt.close()

# Simulation example with learning rate too slow
Most neurons get stuck at zero and do not converge to a selective fixed point. 

In [ ]:
learnrate_ibcm_low = 2e-4
tau_avg_ibcm_low = 1600  # 2000
ibcm_rates_low = [
    learnrate_ibcm_low, 
    tau_avg_ibcm_low, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]

# Test run for now
# Create a default background for testing purposes
meta_seedseq_low = np.random.SeedSequence(0x65be6823817996324d42794007b45e89)

# Package dimensions and back. parameters
back_rates_default = default_background_params(n_components)
dimensions_ibcm = [n_components, n_dimensions, n_i_ibcm]

# Run and analyze simulation derived from the meta seedsequence
all_res = run_analyze_ibcm_one_back_seed(ibcm_rates_low, back_rates_default, inhib_rates_default, 
                        ibcm_options, dimensions_ibcm, meta_seedseq_low,
                        duration_loc=duration, dt_loc=deltat, skp_loc=skp_default, full_returns=True)

gaps_low, specifs_low, hgamvari_low, hgammas_ser_low, ibcm_results_low = all_res

In [ ]:
# Visualize convergence dynamics first
plot_ibcm_results(ibcm_results_low, hgammas_ser_low, skp=skp_default)
plt.show()
plt.close()

## Save simulations

In [ ]:
fname = pj(outputs_folder, "ibcm_convergence_hgammas_examples.npz")
if do_save_outputs:
    np.savez_compressed(fname,
        hgammas_ser_hi=hgammas_ser_hi,
        hgammas_ser_low=hgammas_ser_low,
        hgammas_ser_def=hgammas_ser_def
    )

# Develop main simulations

Simulations are run in the script ``supplementary_scripts/run_ibcm_convergence_analysis.py`` and results saved in ``results/for_plots/convergence/``. 

We run 32 simulation seeds for each combination of rates on a grid of $\mu, \tau_\Theta$ choices, and we do it for $N_\mathrm{B} = 3, 4, 5, 6, 8$ background odors. 

Later, might do simulations at fixed $N_\mathrm{B}$, and perhaps fixed model rates too, for different whiff and blank durations (scale both up or down?)

And do the same for the concentration amplitude, controlled by $c_0$, to show it is equivalent to scaling $\mu$?

In [ ]:
def compute_odor_coverage(specifs):
    nmu, ntau = specifs.shape[:2]
    cov = np.zeros(specifs.shape[:3])
    for i in range(nmu):
        for j in range(ntau):
            for k in range(specifs.shape[2]):
                cov[i, j, k] = np.unique(specifs[i, j, k]).size
    return cov

In [ ]:
def get_vari_specifs(allhvaris, specifs):
    vari_specifs = np.zeros(allhvaris.shape[:4])
    n_mu, n_tau, n_seed, n_i = specifs.shape[:4]
    for i in range(n_mu):
        for j in range(n_tau):
            for k in range(n_seed):
                vari_specifs[i, j, k] = allhvaris[i, j, k][np.arange(n_i), specifs[i, j, k]]
    return vari_specifs

In [ ]:
# Load complete simulations run on the cluster
conv_results = np.load(pj(outputs_folder, "convergence_vs_ibcm_rates_results_3odors.npz"))

In [ ]:
mutau_grid = conv_results["mutau_grid"]
align_gaps = conv_results["align_gaps"]
gamma_specifs = conv_results["gamma_specifs"]
hgamma_varis = conv_results["hgamma_varis"]

In [ ]:
n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau-1, -1, -1):
    gap_mean_line_tau = np.mean(align_gaps[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_tau = np.std(np.mean(align_gaps[:, j, :], axis=2), axis=1, ddof=1)
    ax.plot(mu_range, gap_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, gap_mean_line_tau-gap_std_line_tau, 
                   gap_mean_line_tau+gap_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Alignment gap\n(larger is better)", xscale="log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title=r"$\tau_{\Theta}$", 
          loc="upper left", frameon=False, fontsize=5, bbox_to_anchor=(0.0, 1.05))
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
tau_range = mutau_grid[1, 0, :]
colors = sns.color_palette("ocean", n_colors=n_tau)
for i in range(n_mu-1, -1, -1):
    gap_mean_line_mu = np.mean(align_gaps[i, :, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_mu = np.std(np.mean(align_gaps[i, :, :], axis=2), axis=1, ddof=1)
    ax.plot(tau_range, gap_mean_line_mu, color=colors[i], label=mutau_grid[0, i, 0])
    ax.fill_between(tau_range, gap_mean_line_mu-gap_std_line_mu, 
                   gap_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"Averaging time scale $\tau_{\Theta}$", 
       ylabel="Alignment gap\n(larger is better)", xscale="log")
ax.legend(title=r"$\mu$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

In [ ]:
align_gaps.shape

In [ ]:
# Coverage of odors: fraction of seeds where all 3 odors are covered?
fig, ax = plt.subplots()
# extent: left, right, bottom, top
#im = ax.imshow(np.mean(align_gaps, axis=(2, 3)).T, 
#          extent=(mu_range.min(), mu_range.max(), tau_range.min()*0.01, tau_range.max()*0.01), 
#         cmap="viridis", aspect="auto")
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], np.mean(align_gaps, axis=(2, 3)), cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
fig.colorbar(im, label="Alignment gap")
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
# Plot the variance of the specific c_gamma, mean across neurons and seeds
vari_specifs = get_vari_specifs(hgamma_varis, gamma_specifs)

# Square root from variance to standard deviation
# and normalize by the average alignment gap
stdev_specifs = np.sqrt(vari_specifs) / np.mean(align_gaps, axis=(2,3), keepdims=True)

for j in range(n_tau):
    std_mean_line_tau = np.mean(stdev_specifs[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    std_std_line_tau = np.std(stdev_specifs[:, j, :], axis=(1, 2), ddof=1)
    ax.plot(mu_range, std_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, std_mean_line_tau-std_std_line_tau, 
    #               std_mean_line_tau+std_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Alignment noise", 
       xscale="log", yscale="log")
ax.legend(title=r"$\tau_{\Theta}$", ncol=2, frameon=False)
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
# Plot the variance of the specific c_gamma, mean across neurons and seeds
n_i = gamma_specifs.shape[3]
for i in range(n_mu-1, -1, -1):
    std_mean_line_mu = np.mean(stdev_specifs[i, :, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    std_std_line_mu = np.std(stdev_specifs[i, :, :], axis=(1, 2), ddof=1)
    ax.plot(tau_range, std_mean_line_mu, color=colors[i], label=mutau_grid[0, i, 0])
    #ax.fill_between(tau_range, std_mean_line_mu-std_std_line_mu, 
    #               std_mean_line_mu+std_std_line_mu, color=colors[i], alpha=0.15)
ax.set(xlabel=r"Averaging time $\tau_\Theta$", 
       ylabel=r"Standard dev. of $h_{\gamma,sp}$" "\n(smaller is better)", yscale="log")
ax.legend(title=r"$\mu$", frameon=False, bbox_to_anchor=(1, 1))
plt.show()
plt.close()

In [ ]:
# Coverage of odors: fraction of seeds where all 3 odors are covered?
odor_coverage = compute_odor_coverage(gamma_specifs)
odor_coverage_mean = np.mean(odor_coverage, axis=2)

fig, ax = plt.subplots()
# extent: left, right, bottom, top
#im = ax.imshow(odor_coverage_mean.T, 
#          extent=(mu_range.min(), mu_range.max(), tau_range.min()*0.01, tau_range.max()*0.01), 
#         cmap="viridis", aspect="auto")
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], odor_coverage_mean, cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log", yscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
ax.set_yticks(tau_range)
fig.colorbar(im, label="Average # odors covered")
plt.show()
plt.close()

In [ ]:
n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_line_tau = odor_coverage_mean[:, j]
    coverage_std_tau = np.std(odor_coverage[:, j], axis=1)  # std across seed
    ax.plot(mu_range, coverage_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, coverage_line_tau-coverage_std_tau, 
    #               coverage_line_tau+coverage_std_tau, color=colors[j], alpha=0.0)
ax.axhline(3.0, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Average # odors covered", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="lower left",  frameon=False, ncol=2)
plt.show()
plt.close()

In [ ]:
# Coverage of odors, but now the fraction of seeds where all 3 odors are covered
n_seeds = odor_coverage.shape[2]
odor_coverage_frac_seeds = (odor_coverage == 3).sum(axis=2) / n_seeds

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_frac_line_tau = odor_coverage_frac_seeds[:, j]
    ax.plot(mu_range, coverage_frac_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
ax.axhline(1.0, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Prob. of full coverage", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Analysis for different odor numbers

In [ ]:
# Load complete simulations run on the cluster
n_odors = 8
conv_results = np.load(pj(outputs_folder, f"convergence_vs_ibcm_rates_results_{n_odors}odors.npz"))

mutau_grid = conv_results["mutau_grid"]
align_gaps = conv_results["align_gaps"]
gamma_specifs = conv_results["gamma_specifs"]
hgamma_varis = conv_results["hgamma_varis"]

n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau-1, -1, -1):
    gap_mean_line_tau = np.mean(align_gaps[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_tau = np.std(np.mean(align_gaps[:, j, :], axis=2), axis=1, ddof=1)
    ax.plot(mu_range, gap_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, gap_mean_line_tau-gap_std_line_tau, 
                   gap_mean_line_tau+gap_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Alignment gap\n(larger is better)", xscale="log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title=r"$\tau_{\Theta}$", 
          loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Heatmap
fig, ax = plt.subplots()
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], np.mean(align_gaps, axis=(2, 3)), cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
fig.colorbar(im, label="Alignment gap")
plt.show()
plt.close()

# Coverage of odors: average number of odors covered. 
# Or fraction of seeds where all 3 odors are covered? TODO
odor_coverage = compute_odor_coverage(gamma_specifs)
odor_coverage_mean = np.mean(odor_coverage, axis=2)

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_line_tau = odor_coverage_mean[:, j]
    coverage_std_tau = np.std(odor_coverage[:, j], axis=1)  # std across seed
    #ax.plot(mu_range, coverage_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.plot(mu_range, np.mean(odor_coverage[:, j], axis=1), color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, coverage_line_tau-coverage_std_tau, 
                   coverage_line_tau+coverage_std_tau, color=colors[j], alpha=0.1)
ax.axhline(n_odors, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Average # odors covered", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Coverage of odors, but now the fraction of seeds where all 3 odors are covered
n_seeds = odor_coverage.shape[2]
odor_coverage_frac_seeds = (odor_coverage == n_odors).sum(axis=2) / n_seeds

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_frac_line_tau = odor_coverage_frac_seeds[:, j]
    ax.plot(mu_range, coverage_frac_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
ax.axhline(1.0, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Prob. of full coverage", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()


# Plot the variance of the specific c_gamma, mean across neurons and seeds
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
n_i = gamma_specifs.shape[3]
vari_specifs = get_vari_specifs(hgamma_varis, gamma_specifs)

# Square root from variance to standard deviation
# and normalize by the average alignment gap
stdev_specifs = np.sqrt(vari_specifs) / np.mean(align_gaps, axis=(2,3), keepdims=True)

for j in range(n_tau):
    std_mean_line_tau = np.mean(stdev_specifs[:, j, :], axis=(1, 2))
    std_std_line_tau = np.std(stdev_specifs[:, j, :], axis=(1, 2), ddof=1)
    ax.plot(mu_range, std_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, std_mean_line_tau-std_std_line_tau, 
    #               std_mean_line_tau+std_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Alignment noise", 
       xscale="log", yscale="log")
ax.legend(title=r"$\tau_{\Theta}$", ncol=2, frameon=False)
plt.show()
plt.close()

## Plots as a function of the number of odors

 Alignment gap, average fraction odors covered as a function of number of odors. 
 
 Pick one $\tau$, show lines for different $\mu$? 

In [ ]:
concatenated_align_gaps = []
concatenated_odor_coverages = []
concatenated_hgams_varis = []
nodors_range = np.asarray([3, 4, 5, 6, 8])
f_name_prefix = "convergence_vs_ibcm_rates_results_"
for n in nodors_range:
    try:
        fp = np.load(pj(outputs_folder, f_name_prefix + f"{n}odors.npz"))
    except FileNotFoundError: 
        continue
    else:
        concatenated_align_gaps.append(fp["align_gaps"])
        specifs_n = fp["gamma_specifs"]
        concatenated_odor_coverages.append(compute_odor_coverage(specifs_n))
        vari_specifs = get_vari_specifs(fp["hgamma_varis"], fp["gamma_specifs"])
        concatenated_hgams_varis.append(vari_specifs)
concatenated_align_gaps = np.stack(concatenated_align_gaps)
concatenated_odor_coverages = np.stack(concatenated_odor_coverages)
concatenated_hgams_varis = np.stack(concatenated_hgams_varis)

In [ ]:
chosen_tau_j = 5  # 1600
fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("ocean", n_colors=n_mu)
for i in range(n_mu-1, -1, -1):
    gap_mean_line_mu = np.mean(concatenated_align_gaps[:, i, chosen_tau_j], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_mu = np.std(np.mean(concatenated_align_gaps[:, i, chosen_tau_j], axis=2), axis=1, ddof=1)
    ax.plot(nodors_range, gap_mean_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker="o", ms=3)
    ax.fill_between(nodors_range, gap_mean_line_mu-gap_std_line_mu, 
                   gap_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"Number of odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Alignment gap\n(larger is better)")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$, ($\tau_{\Theta} = " + "{:d}$)".format(int(mutau_grid[1, 0, chosen_tau_j])), 
          loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()


fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
for i in range(n_mu-1, -1, -1):
    coverage_line_mu = np.mean(concatenated_odor_coverages[:, i, chosen_tau_j], axis=1) / nodors_range
    coverage_std_mu = np.std(concatenated_odor_coverages[:, i, chosen_tau_j], axis=1) / nodors_range  # std across seed
    ax.plot(nodors_range, coverage_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker="o", ms=3)
    ax.fill_between(nodors_range, coverage_line_mu-coverage_std_mu, 
                   coverage_line_mu+coverage_std_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"Number of odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Average fraction odors covered")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\tau_{\Theta} = " + "{:d}$)".format(int(mutau_grid[1, 0, chosen_tau_j])), 
          loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

In [ ]:
# Average the standard deviation across seeds and neurons, divide by the alignment gap average
# Axes are indexed [n_odor, n_mu, n_tau, n_seeds, n_neurons, ...]
align_noises_vs_nodor = (np.sqrt(concatenated_hgams_varis)
                         / np.mean(concatenated_align_gaps, axis=3, keepdims=True))

fig, ax = plt.subplots()
fig.set_size_inches(plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1])
mu_range = mutau_grid[0, :, 0]
for i in range(n_mu-1, -1, -1):
    std_mean_line_mu = np.mean(align_noises_vs_nodor[:, i, chosen_tau_j], axis=(1, 2))
    std_std_line_mu = np.std(np.mean(align_noises_vs_nodor[:, i, chosen_tau_j], axis=2), axis=1, ddof=1)
    ax.plot(nodors_range, std_mean_line_mu, color=colors[i], 
            label=str(mutau_grid[0, i, 0]), marker="o", ms=3)
    ax.fill_between(nodors_range, std_mean_line_mu-std_std_line_mu, 
                   std_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"Number of odors $N_\mathrm{B}$ (in $N_\mathrm{S}=25$ dim.)", 
       ylabel="Alignment noise", #," + r"$\sigma[h_\mathrm{sp}] / \langle \Delta h \rangle$",
      yscale="log")
ax.legend(handles=handles, labels=labels, 
          title=r"$\mu$ ($\mathrm{s^{-1}}$)", 
          loc="upper left", bbox_to_anchor=(0.98, 1), frameon=False)

if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_convergence_turbulent_alignment_noise_vs_nodors.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Explore the effect of the strength of turbulence

Note that for a concentration scale $c_0$ (for unit-normed odor vectors), the IBCM convergence time scales as $1/\mu c_0^2$ (see analytical results and notebook ``non-gaussian_convergence.ipynb``) while the $h_\mathrm{sp}$, $h_\mathrm{ns}$ fixed points have scale $1/c_0$ (see appendix, section 4.E, analytical IBCM solutions for $y_1, y_2$). So, we need to adjust the learning rate to keep $\mu c_0^2$ constant. 

We also need to adjust the $\alpha$ rate for $W$ to prevent numerical instabilities arising from larger $h$ values: so, keep $\alpha / c_0$ constant, $\alpha' =\alpha c_0'/c_0$, $\alpha$ scaled proportional to $c_0$.  

## Initial value correction

The other factor influencing initial convergence time is $1/h_0 - 1/h_{\mathrm{saddle}}$, which we want to keep constant. Since $h_{\mathrm{saddle}} \sim 1/c_0$ while $ h_0 \sim m_0 c_0$, we want to make $m_0 \sim 1/c_0^2$ to ensure $h_0 \sim 1/c_0$ as well. 


### Other issue with the Law variant

$\mu$ is divided, in the Law variant of the IBCM model, by $k_{\Theta} + \Theta$, which scales as $h^2$ and thus as $1/c_0^2$. So, for small $c_0$, $\mu$ is divided by a large $\Theta$ after the initial phase, so the learning rate is effectively too low and convergence to the specific fixed point cannot be completed. Likewise, for large $c_0$, the learning rate is not diminished much by the denominator near the fixed point, so there are more fluctuations. We do not compensate this effect, so it is another contribution to the IBCM network's sensitivity to concentration scale. 

### Bottom line issue

The IBCM model becomes specific in response to the difference between the second and third moments of the background; when $c_0$ is too small, the difference between $m_3 \sim c_0^3$ and $\sigma^2 \sim c_0^2$ is of order $c_0^3 - c_0^2$, so a factor of $c_0-1$ remains even when dividing the learning rate by $c_0^2$; this remains small. 


In [ ]:
def adjust_learnrate(back_pms, back_pms_ref, mu_ref):
    """ Return mu to keep mu * <c>**2 constant compared
    to the average concentration entailed by back_pms_ref
    and the learning rate reference mu_ref. """
    avg_conc_ref = mean_turbulent_concs(back_pms_ref).mean()
    avg_conc_sim = mean_turbulent_concs(back_pms).mean()
    return mu_ref * (avg_conc_ref / avg_conc_sim) ** 2

def adjust_minit_scale(back_pms, back_pms_ref, mscale_ref=0.2):
    c0ref = back_pms_ref[4].mean()
    c0 = back_pms[4].mean()
    mscale = mscale_ref * (c0ref / c0)**2
    return mscale

In [ ]:
learnrate_ibcm_turb_base = 0.00125
tau_avg_ibcm_turb = 2000  # 2000
do_adjust_rate = True
ibcm_rates_turb = [
    learnrate_ibcm_turb_base, 
    tau_avg_ibcm_turb, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm
]
inhib_rates_turb = [5e-5, 1e-5]

# Test run for now
# Create a default background for testing purposes
#meta_seedseq_turb = np.random.SeedSequence(0xbaf3feeb1391885d09d21eed643ee07f)
meta_seedseq_turb = np.random.SeedSequence(0x8f380db086f6b8c843bd7b9969a7a2c0).spawn(2)[1]

# Package dimensions and back. parameters
n_components_turb = 3
back_rates_turb = default_background_params(n_components_turb)

# Reference average conc.
back_rates_turb_base = default_background_params(n_components_turb)

# Try changing background rates here if desired
# Concentration scale c0: multiply up-down to change convergence dynamics
# just as well as the learning rate, although with a different scaling. 
back_rates_turb[4][:] = 0.6  # concentration scale, default 0.6
back_rates_turb[5][:] = 0.5  # concentration cutoff relative to c0, default 0.5

back_rates_turb[1][:] = 20.0  # whiff duration, default 500.0  (steps of 10 ms)
back_rates_turb[2][:] = 1.0  # minimum blank duration, default 1.0  (step of 10 ms)
back_rates_turb[3][:] = 200.0  # blank duration, default 800 (steps of 10 ms)

# adjust mu to compensate average conc. constant
if do_adjust_rate:
    learnrate_ibcm_turb = adjust_learnrate(back_rates_turb, 
            back_rates_turb_base, learnrate_ibcm_turb_base)
    print("Adjusted learning rate:", learnrate_ibcm_turb)
    minit_turb = adjust_minit_scale(back_rates_turb, 
            back_rates_turb_base, 0.2)
else:
    learnrate_ibcm_turb = learnrate_ibcm_turb_base
    print("Not adjusted learning rate:", learnrate_ibcm_turb)
    minit_turb = 0.2
ibcm_rates_turb[0] = learnrate_ibcm_turb

dimensions_ibcm = [n_components_turb, n_dimensions, n_i_ibcm]

# Run and analyze simulation derived from the meta seedsequence
all_res = run_analyze_ibcm_one_back_seed(ibcm_rates_turb, back_rates_turb, inhib_rates_turb, 
                        ibcm_options, dimensions_ibcm, meta_seedseq_turb, minit_scale=minit_turb,
                        duration_loc=duration, dt_loc=deltat, skp_loc=skp_default, full_returns=True)

gaps_turb, specifs_turb, hgamvari_turb, hgammas_ser_turb, ibcm_results_turb = all_res

In [ ]:
# Analytical predictions?
conc_ser = ibcm_results_turb[1][:, :, 1]
mean_conc = np.mean(conc_ser)
moments_conc = [
    mean_conc, 
    np.mean((conc_ser - mean_conc)**2),
    np.mean((conc_ser - mean_conc)**3)
]
hspecif, hnonsp, hdpred, u2pred = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components_turb-1)
print(moments_conc)

In [ ]:
# Visualize convergence dynamics first
fig1, ax1, _, _ = plot_ibcm_results(ibcm_results_turb, hgammas_ser_turb, skp=skp_default)
# Plot analytical predictions, because when the time scale is shorter, the average hgammas are
# very close to them; long time scales and ensuing correlations between s, theta, m cause deviations. 
ax1.axhline(hspecif, ls="--", color="k")
ax1.axhline(hnonsp, ls="--", color="grey")
plt.show()
plt.close()

In [ ]:
gaps_turb

# Plot the results of the full simulation scripts

Added main functions to test convergence as a function of $c_0$ vs $\mu$ and of $t_w$ and $t_b$ upper limits. 

In [ ]:
# Load complete simulations run on the cluster
n_odors = 3
conv_results = np.load(pj(outputs_folder, "convergence_vs_turbulence_strength_results_3odors.npz"))

mutau_grid = conv_results["mutau_grid"]
align_gaps = conv_results["align_gaps"]
gamma_specifs = conv_results["gamma_specifs"]
hgamma_varis = conv_results["hgamma_varis"]

n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau-1, -1, -1):
    gap_mean_line_tau = np.mean(align_gaps[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_tau = np.std(np.mean(align_gaps[:, j, :], axis=2), axis=1, ddof=1)
    ax.plot(mu_range, gap_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, gap_mean_line_tau-gap_std_line_tau, 
                   gap_mean_line_tau+gap_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Alignment gap\n(larger is better)", xscale="log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title=r"$\tau_{\Theta}$", 
          loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Heatmap
fig, ax = plt.subplots()
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], np.mean(align_gaps, axis=(2, 3)), cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log", yscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
ax.set_ylim(tau_range[0], tau_range[-1])
fig.colorbar(im, label="Alignment gap")
plt.show()
plt.close()

# Coverage of odors: average number of odors covered. 
# Or fraction of seeds where all 3 odors are covered? TODO
odor_coverage = compute_odor_coverage(gamma_specifs)
odor_coverage_mean = np.mean(odor_coverage, axis=2)

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_line_tau = odor_coverage_mean[:, j]
    coverage_std_tau = np.std(odor_coverage[:, j], axis=1)  # std across seed
    #ax.plot(mu_range, coverage_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.plot(mu_range, np.mean(odor_coverage[:, j], axis=1), color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, coverage_line_tau-coverage_std_tau, 
                   coverage_line_tau+coverage_std_tau, color=colors[j], alpha=0.1)
ax.axhline(n_odors, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Average # odors covered", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Coverage of odors, but now the fraction of seeds where all 3 odors are covered
n_seeds = odor_coverage.shape[2]
odor_coverage_frac_seeds = (odor_coverage == n_odors).sum(axis=2) / n_seeds

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_frac_line_tau = odor_coverage_frac_seeds[:, j]
    ax.plot(mu_range, coverage_frac_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
ax.axhline(1.0, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Prob. of full coverage", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()


# Plot the variance of the specific c_gamma, mean across neurons and seeds
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
n_i = gamma_specifs.shape[3]
vari_specifs = get_vari_specifs(hgamma_varis, gamma_specifs)

# Square root from variance to standard deviation
# and normalize by the average alignment gap
stdev_specifs = np.sqrt(vari_specifs) / np.mean(align_gaps, axis=(2,3), keepdims=True)

for j in range(n_tau):
    std_mean_line_tau = np.mean(stdev_specifs[:, j, :], axis=(1, 2))
    std_std_line_tau = np.std(stdev_specifs[:, j, :], axis=(1, 2), ddof=1)
    ax.plot(mu_range, std_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, std_mean_line_tau-std_std_line_tau, 
    #               std_mean_line_tau+std_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Alignment noise", 
       xscale="log", yscale="log")
ax.legend(title=r"$\tau_{\Theta}$", ncol=2, frameon=False)
plt.show()
plt.close()

In [ ]:
align_gaps.shape

In [ ]:
# Load complete simulations run on the cluster
n_odors = 3
conv_results = np.load(pj(outputs_folder, f"convergence_vs_background_ampli_results_3odors.npz"))

mutau_grid = conv_results["mutau_grid"]
align_gaps = conv_results["align_gaps"]
gamma_specifs = conv_results["gamma_specifs"]
hgamma_varis = conv_results["hgamma_varis"]

n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau-1, -1, -1):
    gap_mean_line_tau = np.mean(align_gaps[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_tau = np.std(np.mean(align_gaps[:, j, :], axis=2), axis=1, ddof=1)
    ax.plot(mu_range, gap_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, gap_mean_line_tau-gap_std_line_tau, 
                   gap_mean_line_tau+gap_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Alignment gap\n(larger is better)", xscale="log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title=r"$\tau_{\Theta}$", 
          loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Heatmap
fig, ax = plt.subplots()
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], np.mean(align_gaps, axis=(2, 3)), cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log", yscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
ax.set_ylim(tau_range[0], tau_range[-1])
fig.colorbar(im, label="Alignment gap")
plt.show()
plt.close()

# Coverage of odors: average number of odors covered. 
# Or fraction of seeds where all 3 odors are covered? TODO
odor_coverage = compute_odor_coverage(gamma_specifs)
odor_coverage_mean = np.mean(odor_coverage, axis=2)

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_line_tau = odor_coverage_mean[:, j]
    coverage_std_tau = np.std(odor_coverage[:, j], axis=1)  # std across seed
    #ax.plot(mu_range, coverage_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.plot(mu_range, np.mean(odor_coverage[:, j], axis=1), color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, coverage_line_tau-coverage_std_tau, 
                   coverage_line_tau+coverage_std_tau, color=colors[j], alpha=0.1)
ax.axhline(n_odors, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Average # odors covered", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

# Coverage of odors, but now the fraction of seeds where all 3 odors are covered
n_seeds = odor_coverage.shape[2]
odor_coverage_frac_seeds = (odor_coverage == n_odors).sum(axis=2) / n_seeds

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_frac_line_tau = odor_coverage_frac_seeds[:, j]
    ax.plot(mu_range, coverage_frac_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
ax.axhline(1.0, ls="--", color="k")
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Prob. of full coverage", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()


# Plot the variance of the specific c_gamma, mean across neurons and seeds
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
n_i = gamma_specifs.shape[3]
vari_specifs = get_vari_specifs(hgamma_varis, gamma_specifs)

# Square root from variance to standard deviation
# and normalize by the average alignment gap
stdev_specifs = np.sqrt(vari_specifs) / np.mean(align_gaps, axis=(2,3), keepdims=True)

for j in range(n_tau):
    std_mean_line_tau = np.mean(stdev_specifs[:, j, :], axis=(1, 2))
    std_std_line_tau = np.std(stdev_specifs[:, j, :], axis=(1, 2), ddof=1)
    ax.plot(mu_range, std_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, std_mean_line_tau-std_std_line_tau, 
    #               std_mean_line_tau+std_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Alignment noise", 
       xscale="log", yscale="log")
ax.legend(title=r"$\tau_{\Theta}$", ncol=2, frameon=False)
plt.show()
plt.close()